In [2]:
import re
import json
import time
import requests
import pandas as pd
from scapy.all import traceroute

In [3]:
def get_re_url(url, expr):
    result = {}
    
    response = requests.get(url)
    
    for k, v in expr.items():
        search_result = re.search(v, response.text)
        
        if search_result != None:
            result[k] = search_result.groups()[0]
        else:
            result[k] = ""
    
    return result

In [8]:
def request_extreme_ip_lookup():
    def wrapped(ip):
        data = get_re_url(f"https://extreme-ip-lookup.com/{ip}", {
            "data": 'URLIP={([^}]+)}'
        })

        obj = json.loads("{" + data["data"] + "}")
    
        return {
            "hostname": obj["ipName"]
            ,"country": obj["country"]
            ,"state": obj["region"]
            ,"city": obj["city"]
            ,"isp": obj["isp"]
            ,"latitude": obj["lat"]
            ,"longitude": obj["lon"]
        }
    
    return wrapped

In [5]:
def request_whatismyipaddress():
    def wrapped(ip):
        data = get_re_url(f"https://whatismyipaddress.com/ip/{ip}", {
            "hostname": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Hostname:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"country": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Country:[ |\t|\n||\r]*<\/th><td>(.*)<img'
            ,"state": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*State\/Region:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"city": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*City:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"isp": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*ISP:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"latitude": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Latitude:[ |\t|\n||\r]*<\/th><td>[ |\t|\n||\r]*(.*)[ |\t|\n||\r]*<\/td>'
            ,"longitude": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Longitude:[ |\t|\n||\r]*<\/th><td>[ |\t|\n||\r]*(.*)[ |\t|\n||\r]*<\/td>'
        })
    
        return data
    
    return wrapped

In [9]:
def traceroute_table(uri, deltatime=1, minttl=1, maxttl=32):
    result, _ = traceroute(uri, minttl=minttl, maxttl=maxttl, verbose=False)

    table = []
    for snd, rcv in result:
        result = request_extreme_ip_lookup()(rcv.src)
        time.sleep(deltatime)

        table.append((
            snd.ttl
            , rcv.src
            , result["hostname"]
            , result["country"]
            , result["state"]
            , result["city"]
            , result["isp"]
            , result["latitude"]
            , result["longitude"]
            , (rcv.time - snd.sent_time)*1000
        ))

    table_df = (
        pd.DataFrame(table, columns=["ttl", "address", "hostname", "country", "state", "city", "isp", "latitude", "longitude", "ms"])
        .sort_values("ttl")
        .drop_duplicates(subset=["address"], keep="first")
    )

    return table_df

In [10]:
traceroute_table("www.cmjornal.pt")

,ttl,address,hostname,country,state,city,isp,latitude,longitude,ms
0,1,192.168.15.1,,,,,,,,1.299143
1,2,179.185.128.209,179.185.128.209.dynamic.adsl.gvt.net.br,Brazil,Sao Paulo,Indaiatuba,Telefonica Brasil S.a,-23.08842,-47.2119,3.582001
2,3,177.16.40.9,177.16.40.9.static.host.gvt.net.br,Brazil,Parana,Curitiba,Telefonica Brasil S.a,-25.42778,-49.27306,1.862764
3,4,152.255.133.70,152-255-133-70.user.vivozap.com.br,Brazil,Sao Paulo,Sao Paulo,Telefonica Brasil S.a,-23.5475,-46.63611,4.196882
4,6,216.184.112.208,grafortw2-1-3-0.net.telefonicaglobalsolutions.com,United States,Florida,Miami,Telxius,25.77427,-80.19366,12.204170
6,9,154.54.11.157,be3017.ccr21.mia03.atlas.cogentco.com,United States,Florida,Miami,African Network Information Center,25.77427,-80.19366,88.444948
5,10,154.54.47.29,be3401.ccr22.mia01.atlas.cogentco.com,United States,Florida,Miami,African Network Information Center,25.77427,-80.19366,81.354380
8,11,154.54.28.49,be3483.ccr42.atl01.atlas.cogentco.com,United States,Georgia,Atlanta,African Network Information Center,33.749,-84.38798,100.085258
7,12,154.54.24.221,be2113.ccr42.dca01.atlas.cogentco.com,United States,Virginia,Arlington,African Network Information Center,38.88101,-77.10428,98.425627
12,13,154.54.61.102,be2313.rcr21.opo01.atlas.cogentco.com,United Kingdom,England,London,African Network Information Center,51.50853,-0.12574,215.019464


In [12]:
traceroute_table("www.polytechnique.edu")

,ttl,address,hostname,country,state,city,isp,latitude,longitude,ms
0,1,192.168.15.1,,,,,,,,0.719547
1,2,179.185.128.209,179.185.128.209.dynamic.adsl.gvt.net.br,Brazil,Sao Paulo,Indaiatuba,Telefonica Brasil S.a,-23.08842,-47.2119,2.481699
2,3,177.16.40.11,177.16.40.11.static.host.gvt.net.br,Brazil,Parana,Curitiba,Telefonica Brasil S.a,-25.42778,-49.27306,3.711224
3,4,152.255.133.70,152-255-133-70.user.vivozap.com.br,Brazil,Sao Paulo,Sao Paulo,Telefonica Brasil S.a,-23.5475,-46.63611,5.006313
4,6,5.53.1.202,,Spain,Madrid,Madrid,Telxius,40.4165,-3.70256,15.697002
5,7,94.142.127.50,,Spain,Madrid,Madrid,Telxius,40.4165,-3.70256,79.119682
6,8,84.16.15.165,,United States,New York,New York,Telxius,40.71427,-74.00597,82.249880
7,9,208.116.240.149,ae5.cr6-mia1.ip4.gtt.net,United States,Georgia,Atlanta,GTT,33.749,-84.38798,84.307909
8,10,213.200.119.214,et-3-3-0.cr4-par7.ip4.gtt.net,United States,Illinois,Chicago,GTT,41.85003,-87.65005,167.256832
10,11,193.51.180.131,,France,Nouvelle-Aquitaine,Pau,Renater,43.3,-0.36667,175.936460
